#### Project:Convilutional Neural Network
Convolutional Neural Network: This is a special kind of Feedforward neural networks often used in solving image related taks and it has been successfuly used for object detection, image recognition and Neural Stlye transfer applications

#### Packages selection
- The first things is to import all the neccesary packages needed for this project

In [3]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

# select GPU when cuda is available
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

#### Settings
- Configure the device
- define all the hyperparameters to be used and needed to be tuned to achive a better accuracy
- Load and explore the data

In [23]:
# device
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

# hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 10
batch_size = 128

# model architecture parameters
num_classes = 10

# dataset -> MNIST
# Note: Transform.Totensor() scale image from 0-1 range

train_dataset = datasets.MNIST(root='data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='data',
                             train=False,
                             transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=False)

# check the dataset
for images, labels in train_loader:
    print("Image Batch dimension", images.shape)
    print("Image labels dimension", labels.shape)
    break

Image Batch dimension torch.Size([128, 1, 28, 28])
Image labels dimension torch.Size([128])


#### Define the architecture of the model such as
- The number of input layers; which is determined by the features of the data
- Number of total hidden layers in the model consist of (Convolution layer, Pooling Layer (Max Pool), Fully connected layer 
- The output layer node units is determined by the intended outcome to achieve
- Here we have two layers convolutional neural networks

In [24]:
"""
Architecture
X -> Conv layer -> ReLU -> MaxPool -> Conv layer -> Relu -> MaxPool -> FC -> Softmax -> y

Note: we use conv2d and MaxPool2d because the input image is 2d image
"""

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()
        
        # 28x28x1 => 28x28x8
        # Convolution layer 1
        self.conv_1 = nn.Conv2d(in_channels=1,
                               out_channels=8,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=1)
        
        # 28x28x8 =>14x14x8
        # MaxPool layer 1
        self.pool_1 = nn.MaxPool2d(kernel_size=(2, 2),
                                  stride=(2, 2),
                                  padding=0)
        
        # 14x14x8 => 14x14x16
        # Convolution layer 2
        self.conv_2 = nn.Conv2d(in_channels=8, 
                               out_channels=16,
                               kernel_size=(3, 3),
                               stride=(1, 1),
                               padding=1)
        
        # 14x14x16 =>7x7x16
        # MaxPool layer 2
        self.pool_2 = nn.MaxPool2d(kernel_size=(2, 2),
                                  stride=(2, 2),
                                  padding=0)
        
        
        # flatten 7x7x16
        # fully connected layer 1
        self.fc_1 = nn.Linear(7*7*16, num_classes)
        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)
        
        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        # flatten the output from the final pool layer before connecting to fully connected layer
        logits = self.fc_1(out.view(-1, 7*7*16)) 
        probas = F.softmax(logits, dim=1)
        return logits, probas
        

#### Loss function and optimizer
- Instantiate the model
- define the specific Loss function to be used either cross entropy, MSELoss, etc
- define the optimization algorithm to be used either SGD, Adam, RMSprop, Momentum etc.

In [25]:
torch.manual_seed(random_seed)
model = ConvNet(num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### compute accuracy
- A function to compute train and test accuracy

In [26]:
def compute_accuracy(model, data_loader):
    correct_prediction, num_examples = 0, 0
    for features, labels in data_loader:
        features = features.to(device)
        labels = labels.to(device)
        outputs, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += labels.size(0)
        correct_prediction += (predicted_labels == labels).sum()
    return correct_prediction.float() / num_examples * 100

#### Training a model requires the following steps¶
- Reset all the gradients to zero (0)
- Make a forward pass (make a prediction)
- Calculate the loss
- Perform back propagation
- Update all the parameters (weight and biases)

In [27]:
start_time = time.time()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)
        
        # Forward and back pass
        outputs, probas = model(features)
        loss = F.cross_entropy(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Logging
        if not i % 50:
            print('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                 %(epoch+1, num_epochs, i, total_step, loss))
            
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' %(
    epoch+1, num_epochs, compute_accuracy(model, train_loader)))
    print('Time elapsed: %.2f min' %((time.time() - start_time) / 60))
print('Total Training Time: %.2f min' %((time.time() - start_time) / 60))

Epoch: 001/010 | Batch 000/469 | Cost: 2.3044
Epoch: 001/010 | Batch 050/469 | Cost: 1.1113
Epoch: 001/010 | Batch 100/469 | Cost: 0.3436
Epoch: 001/010 | Batch 150/469 | Cost: 0.4722
Epoch: 001/010 | Batch 200/469 | Cost: 0.4307
Epoch: 001/010 | Batch 250/469 | Cost: 0.2935
Epoch: 001/010 | Batch 300/469 | Cost: 0.2337
Epoch: 001/010 | Batch 350/469 | Cost: 0.1907
Epoch: 001/010 | Batch 400/469 | Cost: 0.2570
Epoch: 001/010 | Batch 450/469 | Cost: 0.1821
Epoch: 001/010 training accuracy: 94.25%
Time elapsed: 0.85 min
Epoch: 002/010 | Batch 000/469 | Cost: 0.1796
Epoch: 002/010 | Batch 050/469 | Cost: 0.2047
Epoch: 002/010 | Batch 100/469 | Cost: 0.1672
Epoch: 002/010 | Batch 150/469 | Cost: 0.1958
Epoch: 002/010 | Batch 200/469 | Cost: 0.0825
Epoch: 002/010 | Batch 250/469 | Cost: 0.1655
Epoch: 002/010 | Batch 300/469 | Cost: 0.1354
Epoch: 002/010 | Batch 350/469 | Cost: 0.1109
Epoch: 002/010 | Batch 400/469 | Cost: 0.1238
Epoch: 002/010 | Batch 450/469 | Cost: 0.0513
Epoch: 002/010 t

#### Testing/Evaluation

In [28]:
# Print the accuracy
print("Test Accuracy:  %.2f%%" %(compute_accuracy(model, test_loader)))

Test Accuracy:  98.67%
